![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)

# <center> Machine Learning Methods </center>
## <center> Lecture 31 - ICA </center>
### <center> CMT </center>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/MachineLearningMethod/ICA/MainCMT.ipynb)

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
import numpy             as np
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rc('font', **{'size' : 16})
# np.random.seed(1)

Consider the data $\left\{ Y_{i}\right\} _{i=1}^{N}$.  
* Sample mean:
$$\overline{Y}:=\frac{1}{N}\sum_{i=1}^{N}Y_{i}$$
* Normalized mean:
$$\tilde{Y}:=\frac{\overline{Y}-\mu_{\overline{Y}}}{\sigma_{\overline{Y}}}$$

### CMT:

In [3]:
from scipy.stats import norm
from ipywidgets  import interact, IntSlider, Layout

M  = 1000
mY = np.random.binomial(10, .1, size=(1000, M))
def PlotCMT(N):
    vY  = mY[:N,:].mean(axis=0)
    vY -= vY.mean()
    vY /= vY.std()
    
    x   = np.linspace(-4, 4, 10001)
    vP  = norm.pdf(x)
    plt.figure(figsize=(12, 6))
    plt.hist  (vY, bins=40, density=True, color='b', edgecolor='k', alpha=.5, label=r'$\tilde{Y}:=\frac{\overline{Y}-\mu_{\overline{Y}}}{\sigma_{\overline{Y}}}$')
    plt.plot  (x, vP, '--r', lw=3,                                            label=r'$\mathcal{N}\left(x;0,1\right)$')
    plt.title (f'{M} realizations from $\\tilde{{Y}}$\nN={N}')
    plt.xlabel('$x$')
    plt.ylim  ([0, 1])
    plt.legend()
    plt.show  ()
    

nSlider = IntSlider(min=1, max=500, step=1, value=1, layout=Layout(width='80%'))
interact(PlotCMT, N=nSlider);

interactive(children=(IntSlider(value=1, description='N', layout=Layout(width='80%'), max=500, min=1), Output(…

### Plot Gaussianity:

In [4]:
N  = 1000
mS = np.random.rand(2, N)

In [5]:
from sklearn.neighbors import KernelDensity
from ipywidgets        import interact, FloatSlider, Layout

def PlotGaussianity(θ):
    mR = np.array([[np.cos(θ), -np.sin(θ)], 
                   [np.sin(θ),  np.cos(θ)]])

    mX        = mR @ mS
    mX       -= mX.mean(axis=1)[:,None]
    mU, vS, _ = np.linalg.svd(mX / np.sqrt(N), full_matrices=False)
    mZ        = mU @ (mU.T @ mX / vS[:,None])

    x     = np.linspace(-3, 3, 1001)
    oKDE1 = KernelDensity(bandwidth=.12).fit(mZ[0:1,:].T)
    oKDE2 = KernelDensity(bandwidth=.12).fit(mZ[1:, :].T)
    vP1   = np.exp(oKDE1.score_samples(x[:,None]).squeeze())
    vP2   = np.exp(oKDE2.score_samples(x[:,None]).squeeze())
    vPz   = norm.pdf(x)
   
    _, vAxes = plt.subplots(1, 3, figsize=(20, 5))
    vAxes[0].scatter   (mZ[0,:], mZ[1,:], s=50, color=[0,1,0], edgecolor='k')
    vAxes[0].axis      ('equal')
    vAxes[0].set_xlabel('$Z_1$')
    vAxes[0].set_ylabel('$Z_2$')

    vAxes[1].hist      (mZ[0,:], bins=40, density=True, color='b', edgecolor='k', alpha=.25, label=r'$X_1$')
    vAxes[1].plot      (x, vP1, 'b',    lw=3, label=r'$KDE(Z_1)$')
    vAxes[1].plot      (x, vPz,  '--r', lw=2, label=r'$\mathcal{N}\left(z;0,1\right)$')
    vAxes[1].set_xlabel('$Z_1$')
    vAxes[1].legend    (fontsize=12)

    vAxes[2].hist      (mZ[1,:], bins=40, density=True, color='b', edgecolor='k', alpha=.25, label=r'$X_2$')
    vAxes[2].plot      (x, vP2, 'b',   lw=3, label=r'$KDE(Z_2)$')
    vAxes[2].plot      (x, vPz,  '--r', lw=2, label=r'$\mathcal{N}\left(z;0,1\right)$')
    vAxes[2].set_xlabel('$Z_2$')
    vAxes[2].legend    (fontsize=12)
    plt.show           ()

θSlider = FloatSlider(min=0, max=2*np.pi, step=np.pi/20, value=np.pi/4, layout=Layout(width='80%'))
interact(PlotGaussianity, θ=θSlider);

interactive(children=(FloatSlider(value=0.7853981633974483, description='θ', layout=Layout(width='80%'), max=6…